In [ ]:
import tsfresh

In [8]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import math
import sys
import warnings
sys.path.insert(1, '../')
from API.IdealDataInterface import IdealDataInterface

In [9]:
class featureParser:
    def __init__(self, homeList, filename):

        # Make sure the warning is issued every time the user instantiates the class
        warnings.filterwarnings("always", category=UserWarning,
                                module='featureParser')

        # This will be used to search for the reading files in the directory.
        self.homeList = homeList
        
        if len(self.homeList) == 0:
            warnings.warn('Not enough data to make a prediction.')

        self.init_access()
        self.homeData = self._combine(homeList)
        self.filepath = self.saveFile(filename)
    
    def init_access(self):
        folder_path = 'D:\\MInf_data\\sensordata'
        room_folder_path = 'D:\\MInf_data\\room_sensordata'
        self.ideal = IdealDataInterface(folder_path)
        self.room_ideal = IdealDataInterface(room_folder_path)
    
    def _combine(self, homeList):
        final_df = self._parse(homeList[0])
        final_df = final_df.resample('30min').mean()
        # print(final_df[final_df.index >= '2016-12-16'].head())
        for home in homeList[1:]:
            df = self._parse(home)
            df = df.resample('30min').mean()
            # print(df[df.index >= '2016-12-16'].head())
            # merge final_df and df
            final_df = pd.concat([final_df, df]).groupby(level=0).mean()
        return final_df
            
    def _parse(self, home):
        home_data = self.ideal.get(homeid = home, subtype = 'electric-combined')
        home_df = home_data[0]["readings"]
        return home_df

    def saveFile(self, filename):
        compression_opts = dict(method='zip', archive_name=filename+'.csv')
        dirr = 'generated_data/' + filename + '.zip'
        self.homeData.to_csv(dirr, index=False, compression=compression_opts)
        path = os.path.abspath(dirr)
        return path

In [10]:
featParse = featureParser([64, 73], 'blabla')



Found entries for 1592 sensor readings.
Found entries for 4824 sensor readings.


FileNotFoundError: [Errno 2] No such file or directory: 'generated_data/blabla.zip'

In [ ]:
ff = featParse.homeData[featParse.homeData.index >= '2016-12-16']
print(ff.head())

In [ ]:
#ideal.view(homeid=105, subtype='electric-combined')

#ideal.view(homeid=105, subtype='mains')

data = ideal.get(sensorid=['5093c5097', 5276])
#len(data)

#data[0].keys()

#data[0]['subtype']

assert data[0]['subtype'] == 'electric-combined'
ts_hot_water = data[0]['readings']
ts_sink = data[0]['readings']

ts_hot_water = ts_hot_water.resample('5min').mean()
ts_sink = ts_sink.resample('5min').mean()

ts_combined = pd.concat([ts_hot_water, ts_sink], axis=1)
ts_combined /= 10
#ts_combined.head()

fig, ax = plt.subplots(figsize=(12,5.5))
idx = (ts_combined.index >= '2017-05-19') & (ts_combined.index < '2017-05-20')
ts_combined[idx].plot(ax=ax)